In [ ]:
#| default_exp notifyr

In [ ]:
from nbdev.showdoc import *

# Nostr Notifyr

> Using the basic client to make a simple notification bot to alert when a python function is done running or if it fails

## making a decorator that can send messages
We are going to make a decorator that will store a private key in the keychain and send an encrypted message to itself. The private key will also be assigned to the decorated function as an attribute. The user can then log into any type of nostr client that can receive encrypted DMs to get notifications about python processing results or if the function errors out. Other projects like [knockknock](https://github.com/huggingface/knockknock) offer this sort of service, but take more leg work to get an account set up. Being able to randomly generate a Nostr private key makes this quick and painless.

> **Note**: There is some concern about the safety of the current encrypted message implementation. Please do not use this module for any sort of secure communication. You can see a Github issue on the topic [here](https://github.com/nostr-protocol/nips/issues/107)

In [ ]:
#| export

from nostrfastr.client import Client
import time

First let's make a helper function that will send the message

In [ ]:
#| export

def send_nostr_message(notifyr_client: Client, message: str) -> None:
    """a simple function that takes a client and a message and
    sends the message to the client pubkey from the client pubkey
 
    Parameters
    ----------
    notifyr_client : Client
       A client class that will send an encrypted message for us
    message : str
       A message that will be encrypted and sent
    """
    with notifyr_client:
        recipient_pubkey_hex = notifyr_client.public_key.hex()
        event = notifyr_client.event_encrypted_message(recipient_hex=recipient_pubkey_hex,
                                                       message=message)
        notifyr_client.publish_event(event)
    pass


Next we are going to make helper functions to get and set credentials from `keyring`

In [ ]:
#| export

import keyring
from nostrfastr.nostr import PrivateKey, PublicKey

In [ ]:
show_doc(keyring.set_password)

---

### set_password

>      set_password (service_name:str, username:str, password:str)

Set password for the user in the specified service.

In [ ]:
#| export

def set_private_key(notifyr_privkey_hex: str) -> None:
   """Set the private key in the computer keyring

   Parameters
   ----------
   notifyr_privkey_hex : str
       nostr hex private key
   """
   return keyring.set_password(service_name='nostr',
                               username='notifyr',
                               password=notifyr_privkey_hex)

def get_private_key() -> str:
    """get the nostr hex private key from the computer key ring

    Returns
    -------
    str
        nostr hex private key
    """
    return keyring.get_password(service_name='nostr',
                                username='notifyr')

def delete_private_key() -> None:
    """delete the nostr hex private key from the computer key ring.
    This is not used in the decorator function, but may be used
    if need for testing
    """
    return keyring.delete_password(service_name='nostr',
                                   username='notifyr')


Let's test setting and restoring the private key

In [ ]:
#| hide
current_machine_privkey = get_private_key()

In [ ]:
delete_private_key()
assert get_private_key() is None
priv_key_hex = PrivateKey().hex()
set_private_key(notifyr_privkey_hex = priv_key_hex)
assert get_private_key() == priv_key_hex

In [ ]:
#| hide
if current_machine_privkey is not None:
    set_private_key(notifyr_privkey_hex=current_machine_privkey)

Finally we write the decorator function complete with
 - nostr client handling
 - start message, success message, error message handling
 - and setting the private key to the decorator function for easy user access

In [ ]:
#| export

def notifyr(func):
   """A decorator that will set a nostr private key to `func.notifyr_privkey_hex
   and use that key to send an encrypted message to it's own public key on the start
   and termination of the decorated function. The output will send whether the function
   runs completely or ends in an error with an informative message.

   Parameters
   ----------
   func : function
       the function to be decorated

   Returns
   -------
   function
       the decorated function

   Raises
   ------
   e
       if the function fails, else returns the function result
   """

   notifyr_privkey_hex = get_private_key()
   if notifyr_privkey_hex is None:
      notifyr_privkey_hex = PrivateKey.hex()
   set_private_key(notifyr_privkey_hex)
   assert get_private_key() == notifyr_privkey_hex

   def loud_process(*args,**kwargs):
      notifyr_privkey_hex = get_private_key()
      notifyr_client = Client(private_key_hex=notifyr_privkey_hex,
                              relay_urls=['wss://relay.damus.io',
                                          'wss://brb.io'])
      notifyr_pubkey_hex = notifyr_client.public_key.hex()
      function_name = func.__name__
      message = 'process started!'
      send_nostr_message(notifyr_client=notifyr_client,
                         message=message)
      try:
         result = func(*args,**kwargs)
         message = f'**process**: {function_name}\n' \
                   f'**finished** with result of type:\n\t{type(result)}'
      except Exception as e:
         result = e
         message = f'**process name**: {function_name}\n' \
                   f'**failed** with error:\n\t{type(e).__name__}: {e}'
      send_nostr_message(notifyr_client=notifyr_client,
                         message=message)
      if issubclass(type(result), Exception):
         raise result
      else:
         return result
   loud_process.notifyr_private_key = notifyr_privkey_hex
   return loud_process

Now we can decorate a couple functions!

In [ ]:
@notifyr
def raise_error():
    raise Exception('Oh no! Process failed!')

@notifyr
def success():
    return True

And test a successful function

In [ ]:
success()

logged in as public key
	bech32: npub1l9vydnkzgfpcfrtklw30kj0alhcyd3svgayrx26wa8n4tmr0zavq88gayp
	hex: f95846cec24243848d76fba2fb49fdfdf046c60c4748332b4ee9e755ec6f1758


/Users/ryanarmstrong/python/nostrfastr/nostrfastr/client.py:482: UserWarning: the current implementation of messages should be used with caution
                    see https://github.com/nostr-protocol/nips/issues/107
  warnings.warn('''the current implementation of messages should be used with caution


True

And test a failing function

In [ ]:
from fastcore.test import test_fail

In [ ]:
test_fail(raise_error)

logged in as public key
	bech32: npub1l9vydnkzgfpcfrtklw30kj0alhcyd3svgayrx26wa8n4tmr0zavq88gayp
	hex: f95846cec24243848d76fba2fb49fdfdf046c60c4748332b4ee9e755ec6f1758


/Users/ryanarmstrong/python/nostrfastr/nostrfastr/nostr.py:164: UserWarning: wss://brb.io is not connected... removing relay.
  warnings.warn(
/Users/ryanarmstrong/python/nostrfastr/nostrfastr/client.py:482: UserWarning: the current implementation of messages should be used with caution
                    see https://github.com/nostr-protocol/nips/issues/107
  warnings.warn('''the current implementation of messages should be used with caution


Remember we can retrieve the private key to log into our messages from the keychain or from `success.notifyr_privkey_hex` or `raise_error.notifyr_privkey_hex`.

In this case you will either have to trust me that it works... or verify for yourself!

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()